In [94]:
# %load statistics.py
def vector_add(v,w):
    return [v_i + w_i for v_i, w_i in zip(v,w)]

def vector_subtract(v,w):
    return [v_i - w_i for v_i, w_i in zip(v,w)]

def vector_sum(vectors):
    return reduce(vector_add, vectors)

def uniform_pdf(x):
    return 1 if x >= 0 and x < 1 else 0

def uniform_cdf(x):
    if x < 0: return 0
    elif x < 1 : return x
    else: return 1

def normal_pdf(x, mu=0, sigma=1):
    sqrt_two_pi = math.sqrt(2 * math.pi)
    return (1 / sqrt_two_pi * sigma) * ( math.exp( -(x-mu)**2/2*sigma**2 ) )

def normal_cdf(x, mu=0, sigma=1):
    return ( 1+ math.erf((x-mu) / math.sqrt(2) / sigma)) / 2

def inverse_normal_cdf(p, mu=0, sigma=1, tolerance = 0.00001):
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance = tolerance)
    low_z, low_p = -10.0, 0
    hi_z, hi_p = 10.0, 1
    while hi_z - low_z > tolerance:
        mid_z = (hi_z + low_z) / 2
        mid_p = normal_cdf(mid_z)
        if mid_p < p:
            low_Z, low_p = mid_z, mid_p
        elif mid_p > p:
            hi_z, hi_p = mid_Z, mid_p
        else:
            break
    return mid_z


def bernoulli_trial(p):
    return 1 if random.random() < p else 0

def binomial(n, p):
    return sum(bernoulli_trial(p) for _ in range(n))

def normal_probablity_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

def normal_probablity_below(x, mu=0, sigma=1):
    return normal_cdf(x, mu=0, sigma=1)

def normal_probablity_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

def normal_probablity_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probablity_between(lo, hi, mu, sigma)

def normal_upper_bound(probablity, mu=0, sigma=1):
    return inverse_normal_cdf(probablity, mu, sigma)

def normal_lower_bound(probablity, mu=0, sigma=1):
    return inverse_normal_cdf(1-probablity, mu, sigma)

def normal_two_slided_bound(probablity, mu=0, sigma=1):
    tail_probablity = ( 1- probablity) / 2
    upper_bound = normal_lower_bound(tail_probablity, mu, sigma)
    lower_bound = normal_upper_bound(tail_probablity, mu, sigma)
    return lower_bound, upper_bound

def two_sided_p_value(x, mu=0, sigma=1):
    if x > mu:
        return 2 * normal_probablity_above(x, mu, sigma)
    else:
        return 2 * normal_probablity_below(x, mu, sigma)

## for AB Test
def estimated_parameters(N, n):
    p = n / N
    sigma = math.sqrt(p * (1-p) / N) #N이 크기때문에 편의상 t분포가 아닌 정규분포를 이용
    return p, sigma

def ab_test_statistics(Na, na, Nb, nb): 
    #Na Number of A test users, na : Conversion counts of A test
    pA, sigmaA = estimated_parameters(Na, na)
    pB, sigmaB = estimated_parameters(Nb, nb)
    return (pB - pA) / math.sqrt(sigmaA ** 2 + sigmaB ** 2)


In [35]:
import math
def normal_approximation_to_binomial(n,p):
    mu = p *n
    sigma = math.sqrt(p * (1-p) * n)
    return mu, sigma

In [36]:
normal_approximation_below = normal_cdf

In [37]:
def normal_probablity_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

In [38]:
def normal_probablity_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

In [39]:
def normal_probablity_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probablity_between(lo, hi, mu, sigma)

In [40]:
def normal_upper_bound(probablity, mu=0, sigma=1):
    return inverse_normal_cdf(probablity, mu, sigma)

In [41]:
def normal_lower_bound(probablity, mu=0, sigma=1):
    return inverse_normal_cdf(1-probablity, mu, sigma)

In [42]:
def normal_two_slided_bound(probablity, mu=0, sigma=1):
    tail_probablity = ( 1- probablity) / 2
    upper_bound = normal_lower_bound(tail_probablity, mu, sigma)
    lower_bound = normal_upper_bound(tail_probablity, mu, sigma)
    return lower_bound, upper_bound

In [43]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000,0.5)

In [44]:
normal_two_slided_bound(0.95,mu_0, sigma_0)

(469.01026640487555, 530.9897335951244)

In [45]:
lo, hi = normal_two_slided_bound(0.95, mu_0, sigma_0)

In [52]:
print("유의수준 (제1종 오류) : ", round(lo), '~' , round(hi))

유의수준 (제1종 오류) :  469 ~ 531


In [61]:
mu_1, sigma_1 = normal_approximation_to_binomial(1000,0.55)

In [62]:
type2_probablity = normal_probablity_between(lo, hi, mu_1, sigma_1)

In [63]:
power = 1 - type2_probablity
print('power :', power)

power : 0.8865480012953671


In [69]:
two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598857

In [73]:
from random import random
extreme_value_count = 0
for _ in range(100000):
    num_head = sum ( 1 if random() < 0.5 else 0
                   for _ in range(1000))
    if num_head >= 530 or num_head <= 470:
        extreme_value_count += 1

In [74]:
extreme_value_count / 100000

0.06246

In [78]:
two_sided_p_value(531.5, mu_0, sigma_0) ## 귀무가설 기각  p < 0.05

0.046345287837786575

In [88]:
z = ab_test_statistics(1000,200,1000,180)
print(z)

0.2
0.18
-1.1403464899034472


In [96]:
two_sided_p_value(z) # p > 0.05 귀무가설 채택

0.254141976542236

In [97]:
z = ab_test_statistics(1000,200,1000,150)
two_sided_p_value(z)

0.2
0.15


0.003189699706216853